In [61]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set()

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
#import datetime as dt

import time
from datetime import datetime
from datetime import date
from datetime import timedelta
import scipy
import scipy.stats
import pylab
import statsmodels.api as sm
from itertools import cycle

import streamlit as st

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

import statsmodels.tsa.stattools as sts
import statsmodels.graphics.tsaplots as sgt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima.arima import auto_arima
from statsmodels.tsa.stattools import adfuller
from pmdarima.arima import OCSBTest
from arch import arch_model
from scipy.stats.distributions import chi2 

import yfinance
from yahoo_fin.stock_info import get_data
from Time_Series_Functions import get_close_column, get_volume_column, create_ticker_table
from Auto_Time_Series_Functions import get_close_column, get_volume_column, auto_create_ticker_table

#importing created functions
from crypto_data_loader import crypto_data_loader
from CryptoDataset import CryptoDataset
from CryptoModel import CryptoModel
from generate_data import generate_data
from get_preds import get_preds
from standardize_data import standardize_data
from train_val_split import train_val_split
from visualize_results import visualize_results


import warnings
warnings.filterwarnings('ignore')

First, I'm going to focus on the Auto Arima model. 

In order to enhance the automatic capabilities of the function, I will make a few alterations to the <i>create_ticker_table</i> function. Also, I will stick with differencing and logging the data for transformations for now.

In [57]:
def log_and_shift(series):
    series_log = np.log(series)
    series_diff_log = series_log.shift()
    return series_diff_log[1:]

def rev_log_and_shift(series):
    series_rev_exp = np.exp(series.shift(-1))
    series_rev_exp.dropna(axis=0, inplace=True)
    return series_rev_exp

In [64]:
#All Crypto tickers on the first page of cryptocurrencies of Yahoo! Finance
tickers = ['BTC-USD', 'ETH-USD', 'USDT-USD', 'BNB-USD', 'USDC-USD', 'HEX-USD', 'XRP-USD', 'SOL-USD', 'ADA-USD', 'BUSD-USD', 'UST-USD', 'DOGE-USD', 'AVAX-USD', 'DOT-USD', 'SHIB-USD', 'WBTC-USD', 'LUNA1-USD', 'WTRX-USD', 'STETH-USD', 'TRX-USD', 'DAI-USD', 'MATIC-USD', 'NEAR-USD', 'CRO-USD', 'LTC-USD']

In [63]:
df = auto_create_ticker_table()


********** Table Head **********


,BTC-USD_Close,ETH-USD_Close,USDT-USD_Close,BNB-USD_Close,USDC-USD_Close,HEX-USD_Close,XRP-USD_Close,SOL-USD_Close,ADA-USD_Close,BUSD-USD_Close,...,WBTC-USD_Return,LUNA1-USD_Return,WTRX-USD_Return,STETH-USD_Return,TRX-USD_Return,DAI-USD_Return,MATIC-USD_Return,NEAR-USD_Return,CRO-USD_Return,LTC-USD_Return
2022-03-07,38062.039062,2497.771240,1.000123,381.344635,0.999134,0.112310,0.723287,82.130066,0.797379,0.999291,...,-2.898368,-6.400973,0.000000,-4.296704,0.840337,0.010601,-3.971876,-6.744247,-4.285667,-2.487910
2022-03-08,38737.269531,2576.747559,1.000282,381.958557,1.000260,0.109438,0.720977,82.254761,0.801250,1.000158,...,1.731442,9.694246,3.741078,3.248025,3.176546,0.048205,2.041921,0.130453,2.085003,1.768378
2022-03-09,41982.925781,2729.783447,1.000294,393.555756,1.000450,0.136682,0.767236,88.202065,0.849293,1.000157,...,8.410253,16.315416,-0.146487,5.947183,0.490556,-0.010189,4.257501,19.615922,5.627329,6.285625
2022-03-10,39437.460938,2608.048584,1.000565,372.159027,0.999801,0.138977,0.737465,82.939087,0.806316,0.999169,...,-6.280131,2.151697,-1.359859,-4.505081,-1.597611,-0.006400,-4.487608,-4.918648,-4.752458,-4.008322
2022-03-11,38794.972656,2559.562988,1.000200,372.184052,0.999341,0.122838,0.802155,80.744995,0.790125,0.999365,...,-1.371172,-13.160179,0.997611,-1.975373,0.664789,-0.067695,-2.271608,-7.285615,-1.702920,2.250094
2022-03-14,39666.753906,2590.696045,1.000219,373.466156,1.000288,0.112645,0.774656,81.173340,0.803917,1.000959,...,1.976447,7.520025,-0.481471,1.300344,-0.205751,0.121068,-2.033765,-1.054188,-0.907618,0.674792
2022-03-15,39338.785156,2620.149658,1.000334,371.628845,0.999367,0.108333,0.766783,82.955086,0.800006,0.999095,...,-0.464904,-6.555551,0.568588,1.415041,0.595249,-0.109628,0.166698,-0.136866,1.449304,1.244320
2022-03-16,41143.929688,2772.055664,1.000435,385.528229,0.999896,0.115955,0.793538,87.903625,0.838374,0.999472,...,4.349759,0.725739,0.692668,5.314075,0.963615,0.021908,6.387717,6.760644,5.507260,4.085129
2022-03-17,40951.378906,2814.854492,1.000500,391.824738,0.999784,0.116956,0.793503,87.876846,0.836784,0.999899,...,-0.448604,-2.651922,-0.080451,1.851157,-0.469845,-0.019602,-1.310852,-0.781599,-1.153027,-0.941959
2022-03-18,41801.156250,2945.343018,1.000483,397.645508,0.999848,0.117488,0.798266,89.822037,0.853433,0.999178,...,2.121598,1.954251,1.306254,4.563816,1.467181,0.023810,3.481031,5.989851,1.207591,1.636869


********** Table Tail **********


,BTC-USD_Close,ETH-USD_Close,USDT-USD_Close,BNB-USD_Close,USDC-USD_Close,HEX-USD_Close,XRP-USD_Close,SOL-USD_Close,ADA-USD_Close,BUSD-USD_Close,...,WBTC-USD_Return,LUNA1-USD_Return,WTRX-USD_Return,STETH-USD_Return,TRX-USD_Return,DAI-USD_Return,MATIC-USD_Return,NEAR-USD_Return,CRO-USD_Return,LTC-USD_Return
2022-04-19,41502.750000,3104.106445,1.000343,422.387054,0.999897,0.122137,0.775063,108.582031,0.953333,1.000430,...,1.532185,4.986204,3.050875,1.388621,2.935301,0.003100,0.494040,7.497962,1.362673,2.324944
2022-04-20,41374.378906,3077.745850,1.000202,418.354248,0.999847,0.129782,0.752178,105.497002,0.937341,1.000754,...,-0.153248,-0.690085,-0.922604,-1.566897,-1.145688,-0.026604,-0.730443,-4.488412,0.016309,-1.610020
2022-04-21,40527.363281,2987.480713,1.000157,405.021454,1.000282,0.129822,0.733178,101.776138,0.910474,1.000500,...,-2.261466,-4.377331,14.269308,-2.777140,14.072025,-0.003500,-2.671998,-6.299928,-2.251179,-4.448757
2022-04-22,39740.320312,2964.835693,1.000234,406.987518,1.000199,0.127324,0.719194,100.612053,0.907154,0.999258,...,-1.916824,3.175286,-6.894677,-0.255531,-6.531199,0.015801,1.397031,-0.055461,-0.698584,-1.305970
2022-04-25,40458.308594,3009.393555,1.000145,404.350281,1.000218,0.127377,0.694426,101.247635,0.898695,0.999434,...,1.786643,3.579587,-2.174652,1.540431,-2.184462,0.011405,-3.321545,-8.309332,-0.421651,-0.865151
2022-04-26,38117.460938,2808.298340,1.000073,385.483063,1.000127,0.130028,0.641699,95.782150,0.827645,0.999019,...,-5.763877,-8.744470,-5.238087,-6.752669,-5.221103,0.008799,-8.119743,-11.433520,-5.929400,-6.001070
2022-04-27,39241.121094,2888.929688,1.000153,391.445831,1.000321,0.144669,0.652318,98.378868,0.840595,0.999847,...,2.899160,0.609418,1.959486,2.994535,2.101101,-0.017995,1.195006,1.456249,2.736901,2.168676
2022-04-28,39773.828125,2936.940918,1.000191,406.718201,1.000029,0.150283,0.644363,98.185730,0.843950,1.001251,...,1.596607,-0.144152,0.780146,1.574430,0.839915,-0.012204,-1.493295,-3.300346,-0.053898,2.769670
2022-04-29,38609.824219,2815.601807,1.000020,393.062164,1.000023,0.175314,0.611407,94.011345,0.805043,0.998873,...,-3.108774,-4.322516,0.036188,-4.231661,-0.341956,0.016003,-7.936451,-9.197698,-4.974927,-2.850607
2022-05-02,38529.328125,2857.410400,0.999955,389.670654,1.000060,0.175564,0.614269,87.581017,0.781529,0.999786,...,0.101828,-0.948275,7.914443,1.656843,8.271160,-0.055811,-5.522462,4.568276,-25.368553,0.314037


In [44]:
close_cols = df.shape[1] // 3
close_cols

3

In [55]:
new_df = pd.DataFrame()

for j in range(close_cols):
    #print(df.iloc[:,j])
    transformed = log_and_shift(df.iloc[:,j])
    new_df = pd.concat([new_df, transformed], axis=1)

In [58]:
new_df
selection_cols = new_df.columns.tolist()

In [59]:
selection_cols

['BTC-USD_Close', 'LTC-USD_Close', 'DOT-USD_Close']

In [60]:
SYMBOL = st.selectbox(label = "Select equity to analyze",
                      options = selection_cols)

2022-05-10 17:05:34.522 
  command:

    streamlit run C:\Users\Eric\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
bitcoin_diff_log = log_and_shift(crypto_df['BTC-USD_Close'])
df_train, df_test = bitcoin_diff_log[:invasion_date], bitcoin_diff_log[invasion_date:]

litecoin_diff_log = log_and_shift(crypto_df['LTC-USD_Close']) #try passing diff_logs and not crypto_df version
litecoin_train, litecoin_test = litecoin_diff_log[:invasion_date], litecoin_diff_log[invasion_date:]

etherium_diff_log = log_and_shift(crypto_df['ETH-USD_Close'])
etherium_train, etherium_test = etherium_diff_log[:invasion_date], etherium_diff_log[invasion_date:]

lidosteth_diff_log = log_and_shift(crypto_df['STETH-USD_Close'])
lidosteth_train, lidosteth_test = lidosteth_diff_log[:invasion_date], lidosteth_diff_log[invasion_date:]

In [ ]:
exog_data = pd.concat([litecoin_train, etherium_train, lidosteth_train], axis=1)
exog_test = pd.concat([litecoin_test, etherium_test, lidosteth_test], axis=1)

In [ ]:
auto_model = auto_arima(df_train, exogenous = exog_data, #crypto_df['BTC-USD_Close'][:invasion_date], exog_data[1:]
                        start_p = 1, start_q = 1, 
                        test = 'adf',
                        d = None,
                        seasonal = True,
                        start_P = 0,
                        D = 0,
                        trace = True,
                        error_action = 'ignore',
                        suppress_warnings = True,
                        stepwise = True,
                        m = 5, max_q = 5, max_P = 6, max_Q = 6)